In [2]:
import numpy as np

# Blender camera settings
focal_length_mm = 50  # in mm
sensor_width_mm = 36  # in mm (default full-frame)
image_width_px = 1920
image_height_px = 1080

# Convert focal length to pixels
fx = focal_length_mm * (image_width_px / sensor_width_mm)
fy = fx  # square pixels assumed

# Principal point (assuming centre of image)
cx = image_width_px / 2
cy = image_height_px / 2

# Camera intrinsic matrix K
K = np.array([[fx, 0, cx], [0, fy, cy], [0, 0, 1]], dtype=np.float64)

print("Camera Intrinsic Matrix K:")
print(K)

Camera Intrinsic Matrix K:
[[2.66666667e+03 0.00000000e+00 9.60000000e+02]
 [0.00000000e+00 2.66666667e+03 5.40000000e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]


In [5]:
import mediapipe as mp
import cv2
import numpy as np

# Paths
MODEL_PATH = "face_landmarker.task"
front_image_path = "front.png"
left_image_path = "left.png"
right_image_path = "right.png"

# Setup MediaPipe
BaseOptions = mp.tasks.BaseOptions
FaceLandmarker = mp.tasks.vision.FaceLandmarker
FaceLandmarkerOptions = mp.tasks.vision.FaceLandmarkerOptions
VisionRunningMode = mp.tasks.vision.RunningMode

options = FaceLandmarkerOptions(
    base_options=BaseOptions(model_asset_path=MODEL_PATH),
    running_mode=VisionRunningMode.IMAGE,
)


def extract_landmarks(image_path, landmarker):
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=image_rgb)
    result = landmarker.detect(mp_image)

    if not result.face_landmarks:
        raise ValueError(f"No face detected in {image_path}")

    landmarks = result.face_landmarks[0]
    pts = np.array([[lm.x * image.shape[1], lm.y * image.shape[0]] for lm in landmarks])
    return pts


with FaceLandmarker.create_from_options(options) as landmarker:
    pts_front = extract_landmarks(front_image_path, landmarker)
    pts_left = extract_landmarks(left_image_path, landmarker)
    pts_right = extract_landmarks(right_image_path, landmarker)

# Confirm shape
print("Front:", pts_front.shape)
print("Left:", pts_left.shape)
print("Right:", pts_right.shape)

I0000 00:00:1744222922.779275  249321 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M2
W0000 00:00:1744222922.779762  249321 face_landmarker_graph.cc:174] Sets FaceBlendshapesGraph acceleration to xnnpack by default.
W0000 00:00:1744222922.786264  286905 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1744222922.801197  286905 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Front: (478, 2)
Left: (478, 2)
Right: (478, 2)


In [ ]:
triangulated_points = np.full((478, 3), np.nan)
for i in range(478):
    if visible_front[i] and visible_left[i]:
        p3d = triangulate_points(
            P_front, P_left, pts_front[i : i + 1], pts_left[i : i + 1]
        )
    elif visible_front[i] and visible_right[i]:
        p3d = triangulate_points(
            P_front, P_right, pts_front[i : i + 1], pts_right[i : i + 1]
        )
    else:
        continue
    triangulated_points[i] = p3d

NameError: name 'visible_front' is not defined